# Downsampling and BIDS conversion

## Introduction

This section outlines how to prepare the data for further analysis. The initial step involves downsampling the data to an appropriate sampling frequency. Following this, the data will be organised according to the MEG Brain Imaging Data Structure (BIDS) [1]. This ensures the presence of necessary metadata and a consistent file structure across datasets. During BIDS construction, trigger information will also be extracted and labelled.



## Preparation

Import the required modules:

In [1]:
import os
import numpy as np
import pandas as pd
import mne
from mne_bids import (
    BIDSPath,
    make_dataset_description,
    print_dir_tree,
    read_raw_bids,
    write_meg_calibration,
    write_meg_crosstalk,
    write_raw_bids
)

## File overview

The chapter relies on reading the raw FIF-files generated by the acquisition system:
~~~
<ROOT>/Main_split-01_meg.fif
<ROOT>/Main_split-02_meg.fif
~~~ 
the downsampled FIF-files are then generated followed by BIDS data: 
~~~
<ROOT>/Main_fs.fif
<ROOT>/Main_event.fif

<ROOT/Ox_Sub1_BIDS/participants.json
<ROOT/Ox_Sub1_BIDS/participants.tsv
<ROOT/Ox_Sub1_BIDS/sub-01/ses-01/sub-01_ses-01_scans.tsv
<ROOT/Ox_Sub1_BIDS/dataset_description.json
<ROOT/Ox_Sub1_BIDS/sub-01/ses-01/meg/sub-01_ses-01_coordsystem.json
<ROOT/Ox_Sub1_BIDS/ses-01/meg/sub-01_ses-01_task-SpAtt_run-1_channels.tsv
<ROOT/Ox_Sub1_BIDS/ses-01/meg/sub-01_ses-01_task-SpAtt_run-1_events.json
<ROOT/Ox_Sub1_BIDS/meg/sub-01_ses-01_task-SpAtt_run-1_events.tsv
<ROOT/Ox_Sub1_BIDS/meg/sub-01_ses-01_task-SpAtt_run-1_meg.fif
<ROOT/Ox_Sub1_BIDS/sub-01_ses-01_task-SpAtt_run-1_meg.json
~~~

## Downloading and storing the raw data locally

Visit [OpenNeuro](https://openneuro.org/) and download the dataset. TO BE UPDATED 

## Importing the data

The QuSpin OPM data are stored in FIF-format which is a binary file structure with embedded labels. The first step is to import the data by defining the data path where data are downloaded locally. THIS IS USER DEPENDENT. The QuSpin systems break the data up across multiple diles as the total size if a FIF-file cannot exceed 2Gb; however, MNE Python will automatically handle the different sub-files. 
The name of the resampled data will be added 'rs' added in the filename <code>(<a> *_rs.fif </a>)</code>

In [2]:
# The path below is dependent on where the user has stored the data locally
data_path = '/Users/o.jensen@bham.ac.uk/Library/CloudStorage/Dropbox/FLUX/OPM-Oxf/fif'
file_name = '20250410_110557_meg.fif'
raw_fname = os.path.join(data_path, file_name)

raw = mne.io.read_raw_fif(raw_fname, preload=True)
raw_resampled_fname = raw_fname.replace('.fif', f'_rs.fif')

event_fname = raw_fname.replace('.fif', f'_event.fif')

bids_folder = os.path.join(data_path, "Cerca_Spatt_BIDS")

Opening raw data file /Users/o.jensen@bham.ac.uk/Library/CloudStorage/Dropbox/FLUX/OPM-Oxf/fif/20250410_110557_meg.fif...
    Range : 0 ... 2449499 =      0.000 ...  1632.999 secs
Ready.
Opening raw data file /Users/o.jensen@bham.ac.uk/Library/CloudStorage/Dropbox/FLUX/OPM-Oxf/fif/20250410_110557_meg-1.fif...
    Range : 2449500 ... 3079615 =   1633.000 ...  2053.077 secs
Ready.
Reading 0 ... 3079615  =      0.000 ...  2053.077 secs...


To inspect the FIF-data write

In [3]:
print(raw.info)

<Info | 13 non-empty values
 bads: []
 ch_names: Trigger 1, Trigger 2, Trigger 3, Trigger 4, Trigger 5, Trigger ...
 chs: 27 Stimulus, 192 Magnetometers
 custom_ref_applied: False
 dev_head_t: MEG device -> head transform
 device_info: 2 items (dict)
 dig: 15603 items (3 Cardinal, 15600 Extra)
 file_id: 4 items (dict)
 highpass: 0.0 Hz
 line_freq: 0.0
 lowpass: 750.0 Hz
 meas_date: unspecified
 meas_id: 4 items (dict)
 nchan: 219
 projs: []
 sfreq: 1500.0 Hz
>


## Down-sampling the raw data

The next step is to downsample the data to an appropriate sampling frequency. To avoid aliasing artifacts the data must be lowpass filtered at about 1/4 to 1/3 of the desired sampling frequency using a filter with a soft roll-off [2]. The data here will be resampled to 750 Hz after applying a lowpass filtered at 750 Hz/3 = 250 Hz using a finite impulse reponse filter. This will allow for investigating neuronal effects up to 250 Hz which is more than sufficient for most applications. 

In [4]:
desired_sfreq = 750
current_sfreq = raw.info['sfreq']

lowpass_freq = desired_sfreq / 3.0
raw_resampled = raw.copy().filter(l_freq=None, h_freq=lowpass_freq)

raw_resampled.resample(sfreq=desired_sfreq)

Filtering raw data in 1 contiguous segment
Setting up low-pass filter at 2.5e+02 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal lowpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Upper passband edge: 250.00 Hz
- Upper transition bandwidth: 62.50 Hz (-6 dB cutoff frequency: 281.25 Hz)
- Filter length: 81 samples (0.054 s)



[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    2.7s
[Parallel(n_jobs=1)]: Done  71 tasks      | elapsed:   11.3s
[Parallel(n_jobs=1)]: Done 161 tasks      | elapsed:   25.6s


348 events found on stim channel Trigger 5
Event IDs: [1]
348 events found on stim channel Trigger 6
Event IDs: [1]
337 events found on stim channel Trigger 7
Event IDs: [1]
360 events found on stim channel Trigger 9
Event IDs: [1]
309 events found on stim channel Trigger 10
Event IDs: [1]


/var/folders/n0/w7vb4jpn0nx3hs95f4p7r9br0000gr/T/ipykernel_4498/2750611751.py:7: RuntimeWarning: Some events are duplicated in your different stim channels. 325 events were ignored during deduplication.
  raw_resampled.resample(sfreq=desired_sfreq)


348 events found on stim channel Trigger 5
Event IDs: [1]
348 events found on stim channel Trigger 6
Event IDs: [1]
337 events found on stim channel Trigger 7
Event IDs: [1]
360 events found on stim channel Trigger 9
Event IDs: [1]
309 events found on stim channel Trigger 10
Event IDs: [1]


/var/folders/n0/w7vb4jpn0nx3hs95f4p7r9br0000gr/T/ipykernel_4498/2750611751.py:7: RuntimeWarning: Some events are duplicated in your different stim channels. 331 events were ignored during deduplication.
  raw_resampled.resample(sfreq=desired_sfreq)
/var/folders/n0/w7vb4jpn0nx3hs95f4p7r9br0000gr/T/ipykernel_4498/2750611751.py:7: RuntimeWarning: Resampling of the stim channels caused event information to become unreliable. Consider finding events on the original data and passing the event matrix as a parameter.
  raw_resampled.resample(sfreq=desired_sfreq)


<Raw | 20250410_110557_meg.fif, 219 x 1539808 (2053.1 s), ~2.52 GiB, data loaded>

Subsequently, the down-sampled data are stored locally: 

In [5]:
raw_resampled.save(raw_resampled_fname, overwrite=True)

Overwriting existing file.
Writing /Users/o.jensen@bham.ac.uk/Library/CloudStorage/Dropbox/FLUX/OPM-Oxf/fif/20250410_110557_meg_rs.fif


/var/folders/n0/w7vb4jpn0nx3hs95f4p7r9br0000gr/T/ipykernel_4498/649465886.py:1: RuntimeWarning: This filename (/Users/o.jensen@bham.ac.uk/Library/CloudStorage/Dropbox/FLUX/OPM-Oxf/fif/20250410_110557_meg_rs.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw_resampled.save(raw_resampled_fname, overwrite=True)


Closing /Users/o.jensen@bham.ac.uk/Library/CloudStorage/Dropbox/FLUX/OPM-Oxf/fif/20250410_110557_meg_rs.fif
[done]


[PosixPath('/Users/o.jensen@bham.ac.uk/Library/CloudStorage/Dropbox/FLUX/OPM-Oxf/fif/20250410_110557_meg_rs.fif')]

In [6]:
print(raw_resampled.info)

<Info | 13 non-empty values
 bads: []
 ch_names: Trigger 1, Trigger 2, Trigger 3, Trigger 4, Trigger 5, Trigger ...
 chs: 27 Stimulus, 192 Magnetometers
 custom_ref_applied: False
 dev_head_t: MEG device -> head transform
 device_info: 2 items (dict)
 dig: 15603 items (3 Cardinal, 15600 Extra)
 file_id: 4 items (dict)
 highpass: 0.0 Hz
 line_freq: 0.0
 lowpass: 250.0 Hz
 meas_date: unspecified
 meas_id: 4 items (dict)
 nchan: 219
 projs: []
 sfreq: 750.0 Hz
>


The rest of the tutorial will be based on the resampled data. The original FIF-data with the 1500 Hz sample rate can therefore be archived.

## Converting to MEG BIDS format

The next step is to organize the resampled FIF-data according to the BIDS convention. This includes identifying and naming the trigger information. Start by reading the resampled data:

In [7]:
del raw, raw_resampled
raw = mne.io.read_raw(raw_resampled_fname)

Opening raw data file /Users/o.jensen@bham.ac.uk/Library/CloudStorage/Dropbox/FLUX/OPM-Oxf/fif/20250410_110557_meg_rs.fif...
    Range : 0 ... 1539807 =      0.000 ...  2053.076 secs
Ready.


/var/folders/n0/w7vb4jpn0nx3hs95f4p7r9br0000gr/T/ipykernel_4498/1265112872.py:2: RuntimeWarning: This filename (/Users/o.jensen@bham.ac.uk/Library/CloudStorage/Dropbox/FLUX/OPM-Oxf/fif/20250410_110557_meg_rs.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw = mne.io.read_raw(raw_resampled_fname)


### Identifying triggers and write a FIF file

The BIDS data will eventually include the trigger information. The next step is to identify the information in the trigger channels code in the FIF-file. This happens to be from Trigger-line 5 to Trigger-line 10 for the example data, but this is highly system-dependent. Each trigger channel represents a separate digital line created by the computer controlling the experiment feeding into the acquisition system. To reconstruct the events (also referred to as trials), the trigger information from the 7 channels are combined in a single digital trigger channel ('Trigger 1') using a $4^n$ weighting scheme (also highly system dependent), where $n$ is the channel index. The identified events are stored in an additional FIF-file. 

In [8]:
raw.load_data()
trigger_chs =['Trigger 5','Trigger 6','Trigger 7','Trigger 8','Trigger 9','Trigger 10', 'Trigger 11']
trigger_data = raw.copy().pick_channels(trigger_chs).get_data()
n_times = trigger_data.shape[1]
combined_trigger = np.zeros(n_times, dtype=int)

for index_ch, ch_data in enumerate(trigger_data):
    combined_trigger += ch_data.astype(int) * (4 ** (index_ch + 1))
    
trigger1_idx = raw.ch_names.index('Trigger 1')
raw._data[trigger1_idx, :] = combined_trigger
events = mne.find_events(raw, stim_channel='Trigger 1', min_duration=0.003)

# Save events to a .fif file 
mne.write_events(event_fname, events, overwrite=True)

Reading 0 ... 1539807  =      0.000 ...  2053.076 secs...
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
1365 events found on stim channel Trigger 1
Event IDs: [   4   16   64   68   80 1024 1028 1040 4096]
Overwriting existing file.


/var/folders/n0/w7vb4jpn0nx3hs95f4p7r9br0000gr/T/ipykernel_4498/3694027272.py:15: RuntimeWarning: This filename (/Users/o.jensen@bham.ac.uk/Library/CloudStorage/Dropbox/FLUX/OPM-Oxf/fif/20250410_110557_meg_event.fif) does not conform to MNE naming conventions. All events files should end with .eve, -eve.fif, -eve.fif.gz, -eve.lst, -eve.txt, _eve.fif, _eve.fif.gz, _eve.lst or _eve.txt
  mne.write_events(event_fname, events, overwrite=True)


Print the first 10 events for verification

In [9]:
print(events[:10])  

[[87906     0    68]
 [90367     0    16]
 [91270     0    64]
 [91674     0  1040]
 [91963     0  4096]
 [94080     0    16]
 [94982     0    64]
 [95412     0  1040]
 [95725     0  4096]
 [97880     0    16]]


Line 3 of this output demonstrates that at sample-point 182540 there is a trigger with code 64. 
Subsequently, the trigger code from the line Trigger 1 will be assigned labels. This will be dependent on the specific study-design and it is advisable to use informative labels 

In [10]:
event_dict = {
    'off': 0,
    'cue_Right': 4,       
    'cue_Left': 16,       
    'trial_Start': 20,
    'stimOnset': 64,      
    'catchOnset': 1024,     
    'dotOnRight': 1028,
    'dotOnLeft': 1040,
    'resp': 4096,           
    'blkStart': 68,      
    'blkEnd': 80,        
    'expEnd': 260,        
    'abort': 272,
    }

In this example, trigger value 20 denotes the onset off a trial and is therefore labeled 'trial_start'. Likewise 'cue_Right' (trigger 4) and 'cue_Left' (trigger 16) denote respective the displayed cues instructing the participants to attend left or right. 

### Organizing and storing the data according to BIDS

For the BIDS conversion, several parameters must be defined according to the subject and session number. 

In [11]:
raw.info["line_freq"] = 50
raw.set_annotations(None)
subject = '01'
session = '01'
task = 'SpAtt'
run = '01'

bids_path = BIDSPath(
    subject=subject, 
    session=session, 
    task=task, 
    run=run, 
    datatype="meg", 
    root=bids_folder
)
write_raw_bids(
    raw=raw,
    bids_path=bids_path,
    events=event_fname,
    event_id=event_dict,
    overwrite=True,
    allow_preload=True, 
    format='FIF',
    anonymize={'daysback': 50000, 'keep_his': False, 'keep_source': False}
)

Writing '/Users/o.jensen@bham.ac.uk/Library/CloudStorage/Dropbox/FLUX/OPM-Oxf/fif/Cerca_Spatt_BIDS/README'...
Writing '/Users/o.jensen@bham.ac.uk/Library/CloudStorage/Dropbox/FLUX/OPM-Oxf/fif/Cerca_Spatt_BIDS/participants.tsv'...
Writing '/Users/o.jensen@bham.ac.uk/Library/CloudStorage/Dropbox/FLUX/OPM-Oxf/fif/Cerca_Spatt_BIDS/participants.json'...
Writing '/Users/o.jensen@bham.ac.uk/Library/CloudStorage/Dropbox/FLUX/OPM-Oxf/fif/Cerca_Spatt_BIDS/sub-01/ses-01/meg/sub-01_ses-01_coordsystem.json'...
Writing '/Users/o.jensen@bham.ac.uk/Library/CloudStorage/Dropbox/FLUX/OPM-Oxf/fif/Cerca_Spatt_BIDS/sub-01/ses-01/meg/sub-01_ses-01_coordsystem.json'...


/var/folders/n0/w7vb4jpn0nx3hs95f4p7r9br0000gr/T/ipykernel_4498/2775216759.py:16: RuntimeWarning: This filename (/Users/o.jensen@bham.ac.uk/Library/CloudStorage/Dropbox/FLUX/OPM-Oxf/fif/20250410_110557_meg_event.fif) does not conform to MNE naming conventions. All events files should end with .eve, -eve.fif, -eve.fif.gz, -eve.lst, -eve.txt, _eve.fif, _eve.fif.gz, _eve.lst, _eve.txt or -annot.fif
  write_raw_bids(


Used Annotations descriptions: ['blkEnd', 'blkStart', 'catchOnset', 'cue_Left', 'cue_Right', 'dotOnLeft', 'dotOnRight', 'resp', 'stimOnset']
Writing '/Users/o.jensen@bham.ac.uk/Library/CloudStorage/Dropbox/FLUX/OPM-Oxf/fif/Cerca_Spatt_BIDS/sub-01/ses-01/meg/sub-01_ses-01_task-SpAtt_run-01_events.tsv'...
Writing '/Users/o.jensen@bham.ac.uk/Library/CloudStorage/Dropbox/FLUX/OPM-Oxf/fif/Cerca_Spatt_BIDS/sub-01/ses-01/meg/sub-01_ses-01_task-SpAtt_run-01_events.json'...
Writing '/Users/o.jensen@bham.ac.uk/Library/CloudStorage/Dropbox/FLUX/OPM-Oxf/fif/Cerca_Spatt_BIDS/dataset_description.json'...
Writing '/Users/o.jensen@bham.ac.uk/Library/CloudStorage/Dropbox/FLUX/OPM-Oxf/fif/Cerca_Spatt_BIDS/sub-01/ses-01/meg/sub-01_ses-01_task-SpAtt_run-01_meg.json'...
Writing '/Users/o.jensen@bham.ac.uk/Library/CloudStorage/Dropbox/FLUX/OPM-Oxf/fif/Cerca_Spatt_BIDS/sub-01/ses-01/meg/sub-01_ses-01_task-SpAtt_run-01_channels.tsv'...
Copying data files to sub-01_ses-01_task-SpAtt_run-01_meg.fif
Reserving po

BIDSPath(
root: /Users/o.jensen@bham.ac.uk/Library/CloudStorage/Dropbox/FLUX/OPM-Oxf/fif/Cerca_Spatt_BIDS
datatype: meg
basename: sub-01_ses-01_task-SpAtt_run-01_meg.fif)

To explore the data organization according to BIDS inspect the folders: 

In [12]:
print_dir_tree(bids_folder)

|Cerca_Spatt_BIDS/
|--- README
|--- dataset_description.json
|--- participants.json
|--- participants.tsv
|--- sub-01/
|------ ses-01/
|--------- sub-01_ses-01_scans.tsv
|--------- meg/
|------------ sub-01_ses-01_coordsystem.json
|------------ sub-01_ses-01_task-SpAtt_run-01_channels.tsv
|------------ sub-01_ses-01_task-SpAtt_run-01_events.json
|------------ sub-01_ses-01_task-SpAtt_run-01_events.tsv
|------------ sub-01_ses-01_task-SpAtt_run-01_meg.fif
|------------ sub-01_ses-01_task-SpAtt_run-01_meg.json


Some of these files are useful for inspecting the data. For instance 'sub-01_ses-01_task-SpAtt_run-01_events.tsv' is a text-file storing the events and the associated labels. I can be inspected using any text editor. 

**Question 1:**   Inspect the file ''sub-01_ses-01_task-SpAtt_run-01_events.tsv' and report the time and sample points for the first occurences of the 'cue_Left' and 'cue_Right' trigger respectively.

## Preregistration and publication


Publication, example:

"The FIF-data were downsampled to 1500 Hz following the application of a 250 Hz Finite Impulse Rseponse lowpass-filter. Subsequently the data were organized according to MEG-BIDS convention [1]."


## References

[1] Niso G, Gorgolewski KJ, Bock E, Brooks TL, Flandin G, Gramfort A, Henson RN, Jas M, Litvak V, Moreau JT, Oostenveld R, Schoffelen JM, Tadel F, Wexler J, Baillet S. MEG-BIDS, the brain imaging data structure extended to magnetoencephalography. *Scientific Data* 5, 180110 (2018). [doi:10.1038/sdata.2018.110](https://doi.org/10.1038/sdata.2018.110).

[2] Smith SW. *The Scientist and Engineer's Guide to Digital Signal Processing*. California Technical Publishing, 1998. [PDF](https://www.dspguide.com/pdfbook.htm).
